In [ ]:
def evaluate_all_folders(base_path, model, device):
    evaluate = lambda case_path: newmodel.evaluate_case(model, case_path, device)
    evaluations = px.recurse(
        base_path,
        px.is_trainable,
        case_in="  [bold black]{case}.[/bold black] Evaluating...",
        case_out="  ...completed.",
    )(evaluate)
    print("Mean liver score:", sum(item["liver"] for item in evaluations.values()) / len(evaluations))
    print("Mean tumor score:", sum(item["tumor"] for item in evaluations.values()) / len(evaluations))

In [ ]:
### Setup BBE9 #################################################################
model_name = "BBE9_360.pth"
base_path = "/content/drive/MyDrive/COLAB"
drive_mount="/content/drive"
################################################################################

%cd /content/liver
from pathlib import Path

import torch

import newmodel
import utils.path_explorer as px
from newmodel.models import UNet
from utils.namespace import Namespace

env_opts = Namespace(
    dataset_path = Path("/content/dataset"),
    device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"),
    drive_folder = Path(drive_mount) / "MyDrive" / "COLAB",
    models_path = Path(drive_mount) / "MyDrive" / "saved_models",
)

run_opts = Namespace(
    batch_size=5,
    buffer_size=100,
    lr=1e-3 / 20,
    momentum = 0.9,
    resume=False,
    setup_id = "BBE9",
)

net_opts = Namespace(
    channels=[4, 48, 64, 80, 96],
    down_normalization="instance",
    up_dropout="drop",
)

opts = Namespace(env_opts, run_opts, net_opts)

model = UNet(**opts)
model.to(device=opts.device)
model.load_state_dict(torch.load(opts.models_path / model_name, map_location=opts.device))

evaluate_all_folders(Path(base_path), model, opts.device)

In [2]:
# How to download files
# https://drive.google.com/file/d/1oohOGFr-yK9f1AZEwzkzG13-eEBQwn75/view?usp=sharing
# file_download_link = "https://docs.google.com/uc?export=download&confirm=t&id=0B8279de5ddn5RTM1cUZWYW1JQXM"
file_download_link = "https://drive.google.com/drive/folders/0B8279de5ddn5RTM1cUZWYW1JQXM?resourcekey=0-BPxXCfTIxz8PHLyhyubz1g&usp=sharing&export=download&confirm=t&id=0B8279de5ddn5RTM1cUZWYW1JQXM"
!wget -O /content/down.zip --no-check-certificate "$file_download_link"
# https://drive.google.com/drive/folders/0B8279de5ddn5RTM1cUZWYW1JQXM?resourcekey=0-BPxXCfTIxz8PHLyhyubz1g&usp=sharing

--2022-08-06 11:04:28--  https://drive.google.com/drive/folders/0B8279de5ddn5RTM1cUZWYW1JQXM?resourcekey=0-BPxXCfTIxz8PHLyhyubz1g&usp=sharing&export=download&confirm=t&id=0B8279de5ddn5RTM1cUZWYW1JQXM
Resolving drive.google.com (drive.google.com)... 142.250.184.46, 2a00:1450:4002:404::200e
Connecting to drive.google.com (drive.google.com)|142.250.184.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/down.zip’

/content/down.zip       [  <=>               ] 322.80K  1021KB/s    in 0.3s    

2022-08-06 11:04:30 (1021 KB/s) - ‘/content/down.zip’ saved [330544]



In [ ]:
# From trainbundle back to registered images
from pathlib import Path
import numpy as np
import dataset.ndarray as nd

source_path = Path("")
target_path = Path("")
case = ""
case_path = source_path / case
bundle = nd.load_niftidata(case_path)
affine, bottom, top, height = np.eye(4), 0, bundle.shape[3], bundle.shape[3]
regs = {phase: bundle[n] for n, phase in enumerate(["b", "a", "v", "t"])}
nd.save_registereds(regs, target_path / case, affine, bottom, top, height)
nd.save_niftiimage(target_path / case / "segmentation.nii.gz", bundle[4], affine)

In [23]:
# Flip segmentation
from pathlib import Path
import nibabel
import numpy as np
import pickle
import dataset.ndarray

case_path = Path("/mnt/chromeos/GoogleDrive/MyDrive/COLAB") / "Hum249"
orig, orig_aff = dataset.ndarray.load_original(case_path, "v")
print("orig", orig.shape, orig_aff)
reg, reg_aff = dataset.ndarray.load_registered(case_path, "v")
print("reg", reg.shape, reg_aff)
registration_data = pickle.load(open(case_path / "registration_data.pickle", "rb"))
print(registration_data)
segm_image = nibabel.load(case_path / f"segmentation_from_corrected.nii.gz")
segm = np.array(segm_image.dataobj)
segm_aff = segm_image.affine
print("segm", segm.shape, segm_aff)

new_image = nibabel.Nifti1Image(np.flip(segm, axis=1), reg_aff)
# nibabel.save(new_image, case_path / f"segmentation_from_corrected.nii.gz")

orig (512, 512, 219) [[  -0.72070312    0.            0.          187.5       ]
 [  -0.            0.72070312    0.         -341.77929688]
 [   0.           -0.            3.         -128.        ]
 [   0.            0.            0.            1.        ]]
reg (512, 512, 219) [[  -0.72070312    0.            0.          187.5       ]
 [  -0.            0.72070312    0.         -341.77929688]
 [   0.           -0.            3.         -128.        ]
 [   0.            0.            0.            1.        ]]
{'affine': array([[  -0.72070312,    0.        ,    0.        ,  187.5       ],
       [  -0.        ,    0.72070312,    0.        , -341.77929688],
       [   0.        ,   -0.        ,    3.        , -128.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), 'bottom': 1, 'top': 139, 'height': 219}
segm (512, 512, 219) [[  -0.72070312    0.            0.          187.5       ]
 [  -0.            0.72070312    0.         -341.77929688]
 [   0.          

In [1]:
# Fix segmentation
from pathlib import Path
import nibabel
import numpy as np
import pickle
import dataset.ndarray

case_path = Path("/mnt/chromeos/GoogleDrive/MyDrive/COLAB") / "Hum263"
orig, orig_aff = dataset.ndarray.load_original(case_path, "v")
print("orig", orig.shape, orig_aff)
reg, reg_aff = dataset.ndarray.load_registered(case_path, "v")
print("reg", reg.shape, reg_aff)
segm_image = nibabel.load(case_path / f"segmentation_from_drive.nii.gz")
segm = np.array(segm_image.dataobj)
segm_aff = segm_image.affine
print("segm", segm.shape, segm_aff)

registration_data = pickle.load(open(case_path / "registration_data.pickle", "rb"))
print(registration_data)
new_segm = np.zeros_like(reg)
a = registration_data["bottom"]
new_segm[..., a:a+segm.shape[2]] = segm
new_image = nibabel.Nifti1Image(new_segm, reg_aff)
nibabel.save(new_image, case_path / f"segmentation.nii.gz")

orig (512, 512, 86) [[  -0.80078125    0.            0.          220.        ]
 [  -0.            0.80078125    0.         -351.19921875]
 [   0.           -0.            3.         -193.        ]
 [   0.            0.            0.            1.        ]]
reg (512, 512, 86) [[  -0.80078125    0.            0.          220.        ]
 [  -0.            0.80078125    0.         -351.19921875]
 [   0.           -0.            3.         -193.        ]
 [   0.            0.            0.            1.        ]]
segm (512, 512, 85) [[  -0.80078125    0.            0.          220.        ]
 [  -0.            0.80078125    0.         -351.19921875]
 [   0.           -0.            3.         -193.        ]
 [   0.            0.            0.            1.        ]]
{'affine': array([[  -0.80078125,    0.        ,    0.        ,  220.        ],
       [  -0.        ,    0.80078125,    0.        , -351.19921875],
       [   0.        ,   -0.        ,    3.        , -193.        ],
       [   0